In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Intel_images_psnr.zip

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.utils.multiclass import unique_labels

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

In [4]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline
''' to learn more about itertools visit
    https://medium.com/@jasonrigden/a-guide-to-python-itertools-82e5a306cdf8'''
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [5]:
#Import keras functions

from keras import Sequential

'''Since we are using transfer learning let's import the model that we want to implement.Let's use VGG 19(19 layers) and Resnet-50 (50 layers of residual units). 
Residual units allow us to add more layers onto the model without a degradation in accuracy.
Let's try and compare the accuracy of the 2 models and see if the addtional layers do make a significant difference. '''

from keras.applications import VGG19,ResNet50

'Import the datagenerator to augment images'
from keras.preprocessing.image import ImageDataGenerator

'''Import the optimizers and leanring rate annealer (which will reduce the learning rate once a particular metric we choose(in this case validation error) 
does not reduce after a user defined number of epochs)'''
from keras.optimizers import SGD,Adam
from keras.callbacks import ReduceLROnPlateau

'Lastly import the final layers that will be added on top of the base model'
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout

'Import to_categorical from the keras utils package to one hot encode the labels'
from keras.utils import to_categorical

In [51]:
#Data Augmentation Function: Let's define an instance of the ImageDataGenerator class and set the parameters.We have to instantiate for the Train,Validation and Test datasets
train_generator = ImageDataGenerator()

val_generator = ImageDataGenerator()

test_generator = ImageDataGenerator() 

In [52]:
src_path_train = "/content/Intel_images_psnr/train"
src_path_val = "/content/Intel_images_psnr/val"
src_path_test = "/content/Intel_images_psnr/test"

In [53]:
#Fit the augmentation method to the data
train_generatorr = train_generator.flow_from_directory(src_path_train,
                                                    class_mode='categorical',
                                                    batch_size=64,
                                                    color_mode="rgb",
                                                    target_size=(150,150),
                                                    shuffle=False,
                                                    seed=42)

validation_generator = val_generator.flow_from_directory(src_path_val,
                                                        target_size=(150,150),
                                                        batch_size=64,
                                                        color_mode="rgb",
                                                        class_mode='categorical',
                                                        shuffle=False,
                                                        seed=42)
test_generatorr = test_generator.flow_from_directory(
    src_path_test,
    target_size=(150,150),
    color_mode="rgb",
    batch_size=1,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 11224 images belonging to 6 classes.
Found 2810 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [66]:
lrr= ReduceLROnPlateau(verbose = 1,
                       monitor='val_loss', #Metric to be measured
                       factor=.1, #Factor by which learning rate will be reduced
                       patience=5,  #No. of epochs after which if there is no improvement in the val_acc, the learning rate is reduced
                       min_lr=1e-7,
                       min_delta = 0.001) #The minimum learning rate 

In [67]:
base_model_2 = VGG19(include_top=False,weights='imagenet',input_shape=(150,150,3),classes=6)

In [68]:
#Since we have already defined Resnet50 as base_model_2, let us build the sequential model.

model_2=Sequential()
#Add the Dense layers along with activation and batch normalization
model_2.add(base_model_2)
model_2.add(Flatten())


#Add the Dense layers along with activation and batch normalization
model_2.add(Dense(1024,activation=('relu')))
model_2.add(Dropout(0.5))
model_2.add(Dense(512,activation=('relu')))
model_2.add(Dropout(0.5))
model_2.add(Dense(256,activation=('relu')))
model_2.add(Dropout(0.5))
model_2.add(Dense(6,activation=('softmax'))) #This is the classification layer

In [69]:
model_2.layers[0].trainable = False

In [70]:
model_2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten_6 (Flatten)         (None, 8192)              0         
                                                                 
 dense_24 (Dense)            (None, 1024)              8389632   
                                                                 
 dropout_8 (Dropout)         (None, 1024)              0         
                                                                 
 dense_25 (Dense)            (None, 512)               524800    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_26 (Dense)            (None, 256)              

In [71]:
learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.5,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [72]:
model_2.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [73]:
hist = model_2.fit_generator(train_generatorr,
                      epochs=50,
                      steps_per_epoch = train_generatorr.samples // 64,
                      validation_data = validation_generator,
                      validation_steps = validation_generator.samples // 64,
                      callbacks = [lrr],verbose=1)

Epoch 1/50


<ipython-input-73-b774ceaccdde>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model_2.fit_generator(train_generatorr,


175/175 [==============================] - 40s 224ms/step - loss: 5.1376 - accuracy: 0.4024 - val_loss: 0.7942 - val_accuracy: 0.7318 - lr: 0.0010
Epoch 2/50
175/175 [==============================] - 37s 211ms/step - loss: 1.1222 - accuracy: 0.6269 - val_loss: 0.7642 - val_accuracy: 0.7304 - lr: 0.0010
Epoch 3/50
175/175 [==============================] - 37s 210ms/step - loss: 0.8977 - accuracy: 0.6977 - val_loss: 0.9014 - val_accuracy: 0.6846 - lr: 0.0010
Epoch 4/50
175/175 [==============================] - 40s 228ms/step - loss: 0.7879 - accuracy: 0.7343 - val_loss: 0.6258 - val_accuracy: 0.7783 - lr: 0.0010
Epoch 5/50
175/175 [==============================] - 40s 227ms/step - loss: 0.6842 - accuracy: 0.7765 - val_loss: 0.6015 - val_accuracy: 0.8031 - lr: 0.0010
Epoch 6/50
175/175 [==============================] - 40s 227ms/step - loss: 0.5955 - accuracy: 0.8008 - val_loss: 0.6803 - val_accuracy: 0.7838 - lr: 0.0010
Epoch 7/50
175/175 [==============================] - 40s 227ms

KeyboardInterrupt: ignored

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
predict = model_2.evaluate(test_generatorr)
# predict the class label
print(predict)

In [ ]:
model_2.save("model_INTEL_Enhanced_PSNR_VGG.h5")